In [1]:
import pandas as pd
import numpy as np
import pyodbc
con = pyodbc.connect('DSN=DB2v11')


daysback = 30
andWhere = """
and last_test_date > CURRENT_DATE - %d days
and tw.level in ('C5')
and SUBSTR(TRIM(UPPER(dds.calcdefs)),1,4) in ('DI51')
and TRIM(UPPER(dds.corner)) in ('M7125P125AA')
--and tw.family_code not in ('5S','5M')
--and trim(lot_grade) in ('6.1','6.2','6.3','6.4','6.5')
--and lot_id like ('8IQT04004%%')
--and tw.wafer_id in ('65601529SEA5')
""" % (daysback)

pullChipAPRC = """
select
lot_id,
lot_id_base,
wafer_id,
level,
lot_grade,
family_code,
last_test_date,
week,
corner,
sum(bl_failcount_top) as bl_count_top,
sum(bl_failcount_bottom) as bl_count_bottom,
sum(scf_failcount_top) as scf_count_top,
sum(scf_failcount_bottom) as scf_count_bottom,
max(bl_failcount_top) as bl_max_count_top,
max(bl_failcount_bottom) as bl_max_count_bottom,
max(scf_failcount_top) as scf_max_count_top,
max(scf_failcount_bottom) as scf_max_count_bottom,
CASE
  WHEN max(scf_failcount_bottom) > 10 THEN 10
  ELSE max(scf_failcount_bottom)
END AS scf_max_count_bottom_mult,
sum(bl_chipcount_top) as bl_chipcount_top,
sum(bl_chipcount_bottom) as bl_chipcount_bottom,
sum(scf_chipcount_top) as scf_chipcount_top,
sum(scf_chipcount_bottom) as scf_chipcount_bottom,
sum(vpf_chipcount) as vpf_chipcount,
sum(hpf_chipcount) as hpf_chipcount,
sum(unc_chipcount) as unc_chipcount,
sum(pblwlx_chipcount) as pblwlx_chipcount,
sum(lopwl_chipcount) as lopwl_chipcount,
sum(lofwl_chipcount) as lofwl_chipcount,
sum(pwl_chipcount) as pwl_chipcount,
sum(swo_chipcount) as swo_chipcount,
sum(dpf_chipcount) as dpf_chipcount
from
(
    select 
    tw.lot_id,
    substr(tw.lot_id,1,9) as lot_id_base,
    tw.wafer_id,
    tw.level,
    tw.lot_grade,
    tw.family_code,
    tw.last_test_date,
    week(tw.last_test_date) as week,
    tc.wafer_id_xy,
    tc.normalized_testx,
    tc.normalized_testy,
    tc.normalized_testx || '/' || tc.normalized_testy as chip,
    g.geographykey,
    g.radius_center_5,
    CASE
      WHEN g.QUADRANT = '1' THEN 1
      WHEN g.QUADRANT = '2' THEN 1
      ELSE 0
    END AS chip_cnt_top,
    CASE
      WHEN g.QUADRANT = '3' THEN 1
      WHEN g.QUADRANT = '4' THEN 1
      ELSE 0
    END AS chip_cnt_bottom,
    1 AS chip_cnt,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'SCF' AND g.QUADRANT = '1' AND count < 50 THEN count
      WHEN prc.categoryName = 'SCF' AND g.QUADRANT = '2' AND count < 50 THEN count
      ELSE 0
    END AS scf_failcount_top,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'SCF' AND g.QUADRANT = '3' AND count < 50 THEN count
      WHEN prc.categoryName = 'SCF' AND g.QUADRANT = '4' AND count < 50 THEN count
      ELSE 0
    END AS scf_failcount_bottom,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'SCF' AND g.QUADRANT = '1' AND count > 0 AND count < 50 THEN 1
      WHEN prc.categoryName = 'SCF' AND g.QUADRANT = '2' AND count > 0 AND count < 50 THEN 1
      ELSE 0
    END AS scf_chipcount_top,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'SCF' AND g.QUADRANT = '3' AND count > 0 AND count < 50 THEN 1
      WHEN prc.categoryName = 'SCF' AND g.QUADRANT = '4' AND count > 0 AND count < 50 THEN 1
      ELSE 0
    END AS scf_chipcount_bottom,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'LoPBL' AND g.QUADRANT = '1' THEN count
      WHEN prc.categoryName = 'LoPBL' AND g.QUADRANT = '2' THEN count
      ELSE 0
    END AS bl_failcount_top,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'LoPBL' AND g.QUADRANT = '3' THEN count
      WHEN prc.categoryName = 'LoPBL' AND g.QUADRANT = '4' THEN count
      ELSE 0
    END AS bl_failcount_bottom,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'LoPBL' AND g.QUADRANT = '1' AND count > 0 THEN 1
      WHEN prc.categoryName = 'LoPBL' AND g.QUADRANT = '2' AND count > 0 THEN 1
      ELSE 0
    END AS bl_chipcount_top,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'LoPBL' AND g.QUADRANT = '3' AND count > 0 THEN 1
      WHEN prc.categoryName = 'LoPBL' AND g.QUADRANT = '4' AND count > 0 THEN 1
      ELSE 0
    END AS bl_chipcount_bottom,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'VPF' AND count > 0 THEN 1
      ELSE 0
    END AS vpf_chipcount,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'HPF' AND count > 0 THEN 1
      ELSE 0
    END AS hpf_chipcount,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'UNC' AND count > 0 THEN 1
      ELSE 0
    END AS unc_chipcount,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'PBLWLX' AND count > 0 THEN 1
      ELSE 0
    END AS pblwlx_chipcount,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'LoPWL' AND count > 0 THEN 1
      ELSE 0
    END AS lopwl_chipcount,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'LoFWL' AND count > 0 THEN 1
      ELSE 0
    END AS lofwl_chipcount,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'PWL' AND count > 0 THEN 1
      ELSE 0
    END AS pwl_chipcount,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'SWO' AND count > 0 THEN 1
      ELSE 0
    END AS swo_chipcount,
    CASE
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 48 THEN 0
      WHEN tc.normalized_testx = 4 AND tc.normalized_testy = 59 THEN 0
      WHEN tc.normalized_testx = 144 AND tc.normalized_testy = 26 THEN 0
      WHEN tc.normalized_testx = 130 AND tc.normalized_testy = 15 THEN 0
      WHEN prc.categoryName = 'DPF' AND count > 0 THEN 1
      ELSE 0
    END AS dpf_chipcount,
    dds.corner
    from 
    DMIW.PattRecChipFactR prcfr
    join DMIW_SYSTEMS.DERIVEDDATASETUP dds  on prcfr.derivedsetupkey = dds.derivedsetupkey
    join DMIW_SYSTEMS.PATTRECCATEGORY prc   on prcfr.pattRecCatKey = prc.pattRecCatKey
    join DMIW_SYSTEMS.TESTEDWAFER tw        on prcfr.testedwaferkey = tw.testedwaferkey
    join DMIW_SYSTEMS.TESTEDCHIP tc         on prcfr.testedwaferkey = tc.testedwaferkey and prcfr.testedchipkey = tc.testedchipkey
    join DMIW_SYSTEMS.GEOGRAPHY g           on prcfr.geographykey = g.geographykey
    where
    tw.testedwaferkey is not null
    %s
)
group by
lot_id,
lot_id_base,
wafer_id,
level,
lot_grade,
family_code,
last_test_date,
week,
corner
""" % (andWhere)
chipAPRC = pd.read_sql(pullChipAPRC,con)

chipAPRC 

LOT_ID LOT_ID_BASE      WAFER_ID LEVEL LOT_GRADE FAMILY_CODE  \
0     8IQT12005.000   8IQT12005  65506U51SED6    C5                    US   
1     8IQT12005.000   8IQT12005  65506U52SEA5    C5                    US   
2     8IQT12005.000   8IQT12005  65506U53SEE7    C5                    US   
3     8IQT12005.000   8IQT12005  65506U54SEB6    C5                    US   
4     8IQT12005.000   8IQT12005  65506U55SEG0    C5                    US   
5     8IQT12005.000   8IQT12005  65506U5XSEH2    C5                    US   
6     8IQT12005.000   8IQT12005  65506U5YSEE1    C5                    US   
7     8IQT12005.000   8IQT12005  65506U5ZSEB0    C5                    US   
8     8IQT12005.000   8IQT12005  65506U61SEA7    C5                    US   
9     8IQT12005.000   8IQT12005  65506U63SEC0    C5                    US   
10    8IQT12005.021   8IQT12005  65506U65SED1    C5                    US   
11    8IQT12005.000   8IQT12005  65506U66SEA0    C5                    US   
12    8IQT12005.000   8IQT12005  65506U69SEF3    C5                    US   
13    8IQT12005.000   8IQT12005  65506U6ASEC4    C5                    US   
14    8IQT12005.000   8IQT12005  65506U6CSED5    C5                    US   
15    8IQT21013.000   8IQT21013  6560074NSEA7    C5                    9S   
16    8IQT21013.000   8IQT21013  6560074USEB1    C5                    9S   
17    8IQT21013.000   8IQT21013  6560074VSEF3    C5                    9S   
18    8IQT21013.000   8IQT21013  6560074XSEG4    C5                    9S   
19    8IQT21013.000   8IQT21013  6560074YSED3    C5                    9S   
20    8IQT21013.000   8IQT21013  656007B6SEG1    C5                    9S   
21    8IQT21013.000   8IQT21013  656007B8SEH2    C5                    9S   
22    8IQT20010.000   8IQT20010  6560080BSEE6    C5                    US   
23    8IQT20010.014   8IQT20010  6560080DSEF7    C5                    US   
24    8IQT20010.016   8IQT20010  6560080ESEC6    C5                    US   
25    8IQT20010.016   8IQT20010  6560080FSEH0    C5                    US   
26    8IQT20010.014   8IQT20010  6560080GSED7    C5                    US   
27    8IQT20010.014   8IQT20010  6560080ISEF0    C5                    US   
28    8IQT20010.014   8IQT20010  6560080JSEB7    C5                    US   
29    8IQT20010.014   8IQT20010  6560080KSEG1    C5                    US   
...             ...         ...           ...   ...       ...         ...   
1616  8IQT23006.010   8IQT23006  6Q0CZ250SOD4    C5                    US   
1617  8IQT22000.000   8IQT22000  6Q0DE102SOA2    C5                    9S   
1618  8IQT23006.000   8IQT23006  6Q0DE327SOG5    C5                    US   
1619  8IQT23006.000   8IQT23006  6Q0DG113SOC3    C5                    US   
1620  8IQT23006.000   8IQT23006  6Q0DG132SOH1    C5                    US   
1621  8IQT23006.000   8IQT23006  6Q0DG315SOB6    C5                    US   
1622  8IQT22000.000   8IQT22000  6Q0DM078SOC1    C5                    9S   
1623  8IQT23014.012   8IQT23014  6Q0DV118SOH1    C5                    9S   
1624  8IQT23014.000   8IQT23014  6Q0E3309SOC3    C5                    9S   
1625  8IQT23014.000   8IQT23014  6Q0E3311SOC3    C5                    9S   
1626  8IQT20015.000   8IQT20015  6Q0EF122SOF2    C5                    US   
1627  8IQT23004.000   8IQT23004  6Q0IV177SOA5    C5                    US   
1628  8IQT23004.000   8IQT23004  6Q0IX185SOE6    C5                    US   
1629  8IQT22024.000   8IQT22024  6Q0J3027SOF6    C5                    US   
1630  8IQT03008.000   8IQT03008  6Q0J4204SOE6    C5       6.9          5M   
1631  8IQT03008.000   8IQT03008  6Q0J4207SOC6    C5       6.9          5M   
1632  8IQT03008.000   8IQT03008  6Q0J4209SOD7    C5       6.9          5M   
1633  8IQT03008.000   8IQT03008  6Q0J4216SOD0    C5       6.9          5M   
1634  8IQT03008.000   8IQT03008  6Q0J4243SOD6    C5       6.9          5M   
1635  8IQT03008.000   8IQT03008  6Q0J4254SOF1    C5       6.9          5M   
1636  8IQ

In [2]:
df = chipAPRC
conditions = [
    df['VPF_CHIPCOUNT'] > 3,
    df['HPF_CHIPCOUNT'] > 3,
    df['UNC_CHIPCOUNT'] > 3,
    df['PBLWLX_CHIPCOUNT'] > 3,
    df['LOPWL_CHIPCOUNT'] > 3,
    df['LOFWL_CHIPCOUNT'] > 3,
    df['PWL_CHIPCOUNT'] > 3,
    df['SWO_CHIPCOUNT'] > 3,
    df['DPF_CHIPCOUNT'] > 3,
    (df['SCF_CHIPCOUNT_TOP'] == 0) & (df['SCF_CHIPCOUNT_BOTTOM'] > 3),
    df['SCF_CHIPCOUNT_BOTTOM'] > 3
]
choices = [
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    df['SCF_CHIPCOUNT_BOTTOM'] * df['SCF_MAX_COUNT_BOTTOM_MULT'],
    df['SCF_CHIPCOUNT_BOTTOM'] / df['SCF_CHIPCOUNT_TOP'] * df['SCF_MAX_COUNT_BOTTOM_MULT']
]
#(df['SCF_CHIPCOUNT_BOTTOM'] * df['SCF_MEAN_COUNT_BOTTOM'])
df['fin_res_metric'] = np.select(conditions, choices, default=0)
df.to_csv("~/Downloads/finres_30days.csv")